In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
!cp /content/drive/MyDrive/2-folder/kaggle/df_utils.py /content/
import df_utils

In [38]:
def preprocess(df):
    df = df.copy()

    # def normalize_name(x):
    #     return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])

    def ticket_number(x):
        return x.split(" ")[-1]

    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])

    # df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)
    return df

In [41]:
df_train_loaded = pd.read_csv('/content/drive/MyDrive/2-folder/kaggle/titanic/train.csv')
df_test_loaded = pd.read_csv('/content/drive/MyDrive/2-folder/kaggle/titanic/test.csv')
# print(df_train.head(5))

# copy train and test dataframes
df_train = df_train_loaded.copy()
df_test = df_test_loaded.copy()

# extract y values from train set
y = df_train['Survived'].values
# print(y[0:5])
df_train = df_train.drop(['Survived'], axis=1)

# preprocess ticket
df_train = preprocess(df_train)
df_train.Ticket_number = pd.to_numeric(df_train.Ticket_number, errors='coerce')
df_train = df_train.drop(['Ticket'], axis=1)
df_test = preprocess(df_test)
df_test.Ticket_number = pd.to_numeric(df_test.Ticket_number, errors='coerce')
df_test = df_test.drop(['Ticket'], axis=1)
# print(df_train.head(5))

# Drop all columns with with more than 20% of missing values
df_utils.drop_col_miss_val(df_train, df_test, 20)

# Fix categorical values and missing values
df_train, df_test = df_utils.prepare_df(df_train, df_test, 5, ['Survived'])
# print(df_train)

# normalize dataframe
# df_train_x=(df_train_x-df_train_x.mean())/df_train_x.std()

# extract X values from train set
X = df_train.values  #.astype(float)
# print(X[0:5])

# create train, validation and test splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

print(f"X_train = {X_train.shape}")
print(f"y_train = {y_train.shape}")
print(f"X_val = {X_val.shape}")
print(f"y_val = {y_val.shape}")
print(f"X_test = {X_test.shape}")
print(f"y_test = {y_test.shape}")

qqq
['Cabin']
X_train = (534, 12)
y_train = (534,)
X_val = (178, 12)
y_val = (178,)
X_test = (179, 12)
y_test = (179,)
